# Rich H3 Schema — Classification + Elevation Visualization

This notebook reads pre-computed H3 hex data from `duckdb/san_fran_ept_lpc.ddb` 
(built by the 50-min concurrent PDAL pipeline in `new_schema_for_ept_duckdb_h3.ipynb`) 
and produces classification-colored 3D hex maps.

**Tables in the .ddb:**
- `raw_hex_batches` — per-tile H3 aggregates (15 cols, ~3M rows) — the expensive data
- `san_fran_rich_res_13` — final reduced table (re-runnable in seconds from raw_hex_batches)

**NOTE**: Use Jupyter Lab (not VS Code) for lonboard rendering with this much data (~3M hex).

In [1]:
# --- Configuration ---
DB_PATH = 'duckdb/san_fran_ept_lpc.ddb'
DDB_TABLE = 'san_fran_rich_res_13'
H3_RES = 13
MIN_CNT = 10  # filter sparse hex

In [2]:
import duckdb
import numpy as np
import pyarrow.compute as pc

duckdb.sql("INSTALL h3 FROM community")
duckdb.sql("INSTALL spatial")

## Load final table (or re-reduce from raw batches)

If `san_fran_rich_res_13` already exists, just prints summary stats.
Set `FORCE_REBUILD = True` to re-run the reduction from `raw_hex_batches` (~5 sec).

**Geometric structure detection**: CA_SanFrancisco_1_B23 has NO building (class 6) or 
vegetation (class 3/4/5) classification — 69% is class 1 (Unclassified), 30% class 2 (Ground). 
So `building_cnt` and `veg_cnt` from the worker are always 0. Instead, we derive structure class 
from **height above ground** (avg_z - ground_z > 3m) and split building vs vegetation using 
**multi-return ratio** (vegetation has high multi-return ~0.63 from laser penetrating canopy; 
buildings reflect cleanly ~0.14).

In [ ]:
FORCE_REBUILD = False

con = duckdb.connect(DB_PATH)
con.sql("LOAD h3")

table_exists = con.sql(f"""
    SELECT COUNT(*) AS n FROM information_schema.tables
    WHERE table_name = '{DDB_TABLE}'
""").fetchone()[0] > 0

if not table_exists or FORCE_REBUILD:
    print("Rebuilding from raw_hex_batches..." if table_exists else "Table missing, building...")
    con.sql(f"""
        CREATE OR REPLACE TABLE {DDB_TABLE} AS
        WITH base AS (
            SELECT 
                hex,
                h3_cell_to_lat(hex) AS lat,
                h3_cell_to_lng(hex) AS lng,
                SUM(avg_z * cnt) / SUM(cnt) AS avg_z,
                MIN(min_z) AS min_z,
                MAX(max_z) AS max_z,
                MAX(max_z) - MIN(min_z) AS z_range,
                SUM(cnt) AS cnt,
                SUM(ground_z * ground_cnt) / NULLIF(SUM(ground_cnt), 0) AS ground_z,
                SUM(ground_cnt) AS ground_cnt,
                (SUM(avg_z * cnt) / SUM(cnt)) - (SUM(ground_z * ground_cnt) / NULLIF(SUM(ground_cnt), 0)) AS structure_height,
                SUM(building_cnt) AS building_cnt,
                SUM(bridge_cnt) AS bridge_cnt,
                SUM(veg_cnt) AS veg_cnt,
                SUM(water_cnt) AS water_cnt,
                SUM(avg_intensity * cnt) / SUM(cnt) AS avg_intensity,
                SUM(multi_return_ratio * cnt) / SUM(cnt) AS multi_return_ratio,
                SUM(canopy_height * veg_cnt) / NULLIF(SUM(veg_cnt), 0) AS canopy_height
            FROM raw_hex_batches
            GROUP BY 1
        )
        SELECT *,
            CASE
                WHEN bridge_cnt::DOUBLE / cnt > 0.1 THEN 'bridge'
                WHEN water_cnt::DOUBLE / cnt > 0.3 THEN 'water'
                WHEN ground_z IS NOT NULL
                     AND (avg_z - ground_z) > 3.0
                     AND (avg_z - ground_z) <= 40.0
                     AND multi_return_ratio > 0.3 THEN 'vegetation'
                WHEN ground_z IS NOT NULL
                     AND (avg_z - ground_z) > 3.0 THEN 'building'
                ELSE 'ground'
            END AS dominant_class
        FROM base
    """)
    print("Done.")
else:
    print(f"Table {DDB_TABLE} exists, skipping rebuild. Set FORCE_REBUILD = True to re-reduce.")

con.sql(f"SELECT COUNT(*) AS total_hex FROM {DDB_TABLE}").show()
con.sql(f"""
    SELECT dominant_class, COUNT(*) AS hex_count 
    FROM {DDB_TABLE} GROUP BY 1 ORDER BY 2 DESC
""").show()
con.close()

## Viz 1: Simple elevation map (Inferno by max_z)
All hex, single layer, single colormap. Sanity check that the data looks right.

In [4]:
# --- Viz 1: Simple elevation map (Inferno by max_z) ---
# Sanity check — all hex, single layer, single colormap. Uncomment to run.
#
# from lonboard import Map, H3HexagonLayer
# from arro3.core import Table
# from lonboard.colormap import apply_continuous_cmap
# from palettable.matplotlib import Inferno_20
# from matplotlib.colors import Normalize
#
# con = duckdb.connect(DB_PATH, read_only=True)
# con.sql("LOAD h3")
# df = con.sql(f"""
#     SELECT h3_h3_to_string(hex) AS hex, max_z, cnt
#     FROM {DDB_TABLE}
#     WHERE cnt > {MIN_CNT}
# """).fetch_arrow_table()
# con.close()
#
# table = Table.from_arrow(df)
# max_z = np.nan_to_num(np.array(pc.fill_null(df['max_z'], 0)), nan=0)
# norm = Normalize(vmin=max_z.min(), vmax=np.percentile(max_z, 99), clip=True)
# colors = apply_continuous_cmap(norm(max_z), Inferno_20)
#
# layer = H3HexagonLayer(
#     table,
#     get_hexagon=table['hex'],
#     get_fill_color=colors,
#     extruded=True,
#     get_elevation=max_z,
#     elevation_scale=3,
#     stroked=False,
#     opacity=1,
#     coverage=1,
# )
#
# Map(
#     layers=[layer],
#     view_state={'longitude': -122.44, 'latitude': 37.76, 'zoom': 12, 'pitch': 60, 'bearing': 30},
# )

## Viz 2: Classification + height — subdued cmaps, ground opaque

- **Ground + Water**: LaJolla by `ground_z`, alpha 255 (fully opaque base layer)
- **Building**: Teal_7_r by `structure_height`
- **Vegetation**: solid pale green
- **Bridge**: solid gold

Alpha: ground/water = 255, structures/veg/bridge much lower (~100) so the ground reads as terrain.

In [ ]:
from lonboard import Map, H3HexagonLayer
from arro3.core import Table
from lonboard.colormap import apply_continuous_cmap
from palettable.scientific.sequential import LaJolla_20
from palettable.cartocolors.sequential import Teal_7_r
from matplotlib.colors import Normalize

# --- Load data ---
con = duckdb.connect(DB_PATH, read_only=True)
con.sql("LOAD h3")
df = con.sql(f"""
    SELECT h3_h3_to_string(hex) AS hex, avg_z, ground_z, structure_height, dominant_class, cnt
    FROM {DDB_TABLE}
    WHERE cnt > {MIN_CNT}
""").fetch_arrow_table()
con.close()

table = Table.from_arrow(df)
n = len(df)

# --- Extract arrays ---
avg_z = np.nan_to_num(np.array(pc.fill_null(df['avg_z'], 0)), nan=0)
ground_z = np.nan_to_num(np.array(pc.fill_null(df['ground_z'], 0)), nan=0)
struct_h = np.clip(np.nan_to_num(np.array(pc.fill_null(df['structure_height'], 0)), nan=0), 0, None)
classes = np.asarray(df.column('dominant_class')).astype(str)
cnt = np.array(df.column('cnt'))

# --- Extrusion: ground/water by ground_z, structures by avg_z ---
elev = np.where((classes == 'ground') | (classes == 'water'), ground_z, avg_z)

# --- Normalizers ---
ground_pos = ground_z[ground_z > 0]
norm_elev = Normalize(
    vmin=np.percentile(ground_pos, 1) if len(ground_pos) else 0,
    vmax=np.percentile(ground_pos, 99) if len(ground_pos) else 1,
    clip=True
)
struct_pos = struct_h[struct_h > 0]
norm_struct = Normalize(vmin=0, vmax=np.percentile(struct_pos, 95) if len(struct_pos) else 1, clip=True)

# --- Per-class coloring ---
colors_rgb = np.zeros((n, 3), dtype=np.uint8)

# Ground + Water: LaJolla by ground_z
m_gw = (classes == 'ground') | (classes == 'water')
if m_gw.any():
    colors_rgb[m_gw] = apply_continuous_cmap(norm_elev(ground_z[m_gw]), LaJolla_20)

# Building: Teal_7_r by structure_height
m_bld = classes == 'building'
if m_bld.any():
    colors_rgb[m_bld] = apply_continuous_cmap(norm_struct(struct_h[m_bld]), Teal_7_r)

# Vegetation: solid pale green
m_veg = classes == 'vegetation'
if m_veg.any():
    colors_rgb[m_veg] = [144, 190, 109]

# Bridge: solid gold
m_brg = classes == 'bridge'
if m_brg.any():
    colors_rgb[m_brg] = [218, 165, 27]

# --- Alpha: ground/water = 255, everything else = 100 ---
alpha = np.full(n, 100, dtype=np.uint8)
alpha[m_gw] = 255

# --- Combine RGBA ---
colors = np.concatenate([colors_rgb, alpha.reshape(-1, 1)], axis=1)

print(f"{n:,} hex | Classes: {dict(zip(*np.unique(classes, return_counts=True)))}")
print(f"Struct height p95: {np.percentile(struct_pos, 95):.1f}m | Ground elev range: [{norm_elev.vmin:.1f}, {norm_elev.vmax:.1f}]m")

layer = H3HexagonLayer(
    table,
    get_hexagon=table['hex'],
    get_fill_color=colors,
    extruded=True,
    get_elevation=elev,
    elevation_scale=3,
    stroked=False,
    coverage=1,
)

Map(
    layers=[layer],
    view_state={'longitude': -122.44, 'latitude': 37.76, 'zoom': 12, 'pitch': 60, 'bearing': 30},
)

## Viz 3: ASPRS classification colors (flat)
Standard LiDAR class colors — quick check that dominant_class assignment looks reasonable.

In [6]:
# --- Viz 3: ASPRS classification colors (flat) ---
# Quick check that dominant_class assignment looks reasonable. Uncomment to run.
#
# from lonboard import Map, H3HexagonLayer
# from arro3.core import Table
#
# con = duckdb.connect(DB_PATH, read_only=True)
# con.sql("LOAD h3")
# df = con.sql(f"""
#     SELECT h3_h3_to_string(hex) AS hex, max_z, dominant_class, cnt
#     FROM {DDB_TABLE}
#     WHERE cnt > {MIN_CNT}
# """).fetch_arrow_table()
# con.close()
#
# table = Table.from_arrow(df)
# max_z = np.nan_to_num(np.array(pc.fill_null(df['max_z'], 0)), nan=0)
# classes = np.asarray(df.column('dominant_class')).astype(str)
#
# # Colorblind-safe classification colors (no red-green distinction)
# CLASS_COLORS = {
#     'ground':     [186, 148, 86, 255],   # brown
#     'building':   [230, 159, 0, 255],     # orange (colorblind-safe, replaces red)
#     'vegetation': [0, 114, 178, 255],     # blue (colorblind-safe, replaces green)
#     'water':      [86, 180, 233, 255],    # sky blue
#     'bridge':     [160, 160, 160, 255],   # gray
# }
# DEFAULT_COLOR = [200, 200, 200, 255]
# colors = np.array([CLASS_COLORS.get(c, DEFAULT_COLOR) for c in classes], dtype=np.uint8)
#
# layer = H3HexagonLayer(
#     table,
#     get_hexagon=table['hex'],
#     get_fill_color=colors,
#     extruded=True,
#     get_elevation=max_z,
#     elevation_scale=3,
#     stroked=False,
#     opacity=1,
#     coverage=1,
# )
#
# Map(
#     layers=[layer],
#     view_state={'longitude': -122.44, 'latitude': 37.76, 'zoom': 12, 'pitch': 60, 'bearing': 30},
# )

---
## Reference: PDAL Pipeline (already ran — ~50 min)

The cells below are the pipeline that built `raw_hex_batches`. 
**Do NOT re-run** unless you want to rebuild from scratch. The data is in the .ddb file.

In [7]:
# --- Pipeline Configuration (reference only) ---
EPT_URL = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/CA_SanFrancisco_1_B23/ept.json"
SRC_CRS = 'EPSG:3857'
DST_CRS = 'EPSG:4326'
TILE_ZOOM = 16
MAX_WORKERS = 14
SUB_RESOLUTION = None
BBOX = [-13638426.0, 4536715.0, -13617318.0, 4556481.0]

In [8]:
# def get_con():
#     """In-memory connection for workers. LOAD only, no INSTALL."""
#     con = duckdb.connect()
#     con.sql("""
#         SET temp_directory = './tmp';
#         SET memory_limit = '512MB';
#         LOAD h3;
#         LOAD httpfs;
#         LOAD spatial;
#         SET enable_progress_bar = false;
#     """)
#     return con
#
# def process_tile_to_h3(tile):
#     """Worker: PDAL Points -> Rich H3 Aggregates -> Returns Arrow Table"""
#     tb = mercantile.xy_bounds(tile)
#     bounds = f"([{tb.left},{tb.right}],[{tb.bottom},{tb.top}])"
#     reader_opts = {"filename": EPT_URL, "bounds": bounds}
#     if SUB_RESOLUTION:
#         reader_opts["resolution"] = SUB_RESOLUTION
#     try:
#         pipeline = pdal.Reader.ept(**reader_opts).pipeline()
#         count = pipeline.execute()
#         if count == 0 or len(pipeline.arrays) == 0:
#             return None
#         arr = pipeline.arrays[0]
#         if len(arr) == 0: return None
#         arrow_tbl = pa.Table.from_arrays(
#             [
#                 pa.array(arr['X']), pa.array(arr['Y']), pa.array(arr['Z']),
#                 pa.array(arr['Classification']),
#                 pa.array(arr['Intensity']),
#                 pa.array(arr['ReturnNumber']),
#                 pa.array(arr['NumberOfReturns']),
#             ],
#             names=['X', 'Y', 'Z', 'Classification', 'Intensity', 'ReturnNumber', 'NumberOfReturns']
#         )
#         con = get_con()
#         con.register('tile_data', arrow_tbl)
#         hex_summary = con.sql(f"""
#             SELECT 
#                 h3_latlng_to_cell(
#                     ST_Y(ST_Transform(ST_Point(X, Y), '{SRC_CRS}', '{DST_CRS}', always_xy := true)), 
#                     ST_X(ST_Transform(ST_Point(X, Y), '{SRC_CRS}', '{DST_CRS}', always_xy := true)), 
#                     {H3_RES}
#                 ) AS hex,
#                 AVG(Z) AS avg_z, MIN(Z) AS min_z, MAX(Z) AS max_z,
#                 MAX(Z) - MIN(Z) AS z_range, COUNT(*) AS cnt,
#                 AVG(Z) FILTER (WHERE Classification = 2) AS ground_z,
#                 COUNT(*) FILTER (WHERE Classification = 2) AS ground_cnt,
#                 COUNT(*) FILTER (WHERE Classification = 6) AS building_cnt,
#                 COUNT(*) FILTER (WHERE Classification = 17) AS bridge_cnt,
#                 COUNT(*) FILTER (WHERE Classification IN (3,4,5)) AS veg_cnt,
#                 COUNT(*) FILTER (WHERE Classification = 9) AS water_cnt,
#                 AVG(Intensity) AS avg_intensity,
#                 COUNT(*) FILTER (WHERE NumberOfReturns > 1)::DOUBLE / NULLIF(COUNT(*), 0) AS multi_return_ratio,
#                 AVG(Z) FILTER (WHERE ReturnNumber = 1) - AVG(Z) FILTER (WHERE ReturnNumber = NumberOfReturns AND NumberOfReturns > 1) AS canopy_height
#             FROM tile_data
#             GROUP BY 1
#         """).fetch_arrow_table()
#         con.unregister('tile_data')
#         con.close()
#         return hex_summary
#     except Exception as e:
#         print(f"  TILE FAILED {tile}: {e}")
#         return None

In [9]:
# def run_pipeline():
#     import os, time, mercantile, pyarrow as pa, pdal
#     os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
#     sw = mercantile.lnglat(BBOX[0], BBOX[1])
#     ne = mercantile.lnglat(BBOX[2], BBOX[3])
#     tiles = list(mercantile.tiles(sw.lng, sw.lat, ne.lng, ne.lat, zooms=[TILE_ZOOM]))
#     con = duckdb.connect(DB_PATH)
#     con.sql("""
#         CREATE OR REPLACE TABLE raw_hex_batches (
#             hex UBIGINT,
#             avg_z DOUBLE, min_z DOUBLE, max_z DOUBLE, z_range DOUBLE, cnt BIGINT,
#             ground_z DOUBLE, ground_cnt BIGINT,
#             building_cnt BIGINT, bridge_cnt BIGINT, veg_cnt BIGINT, water_cnt BIGINT,
#             avg_intensity DOUBLE, multi_return_ratio DOUBLE, canopy_height DOUBLE
#         )
#     """)
#     con.close()
#     print(f"Processing {len(tiles)} tiles with {MAX_WORKERS} workers...")
#     start = time.time()
#     FLUSH_EVERY = 50
#     pending_tables = []
#     def flush_to_db(tables):
#         if not tables: return
#         combined = pa.concat_tables(tables)
#         con = duckdb.connect(DB_PATH)
#         con.sql("INSERT INTO raw_hex_batches SELECT * FROM combined")
#         con.close()
#     with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
#         futures = {executor.submit(process_tile_to_h3, t): t for t in tiles}
#         for i, future in enumerate(concurrent.futures.as_completed(futures)):
#             result = future.result()
#             if result: pending_tables.append(result)
#             if len(pending_tables) >= FLUSH_EVERY:
#                 flush_to_db(pending_tables)
#                 pending_tables = []
#             if i % 100 == 0:
#                 print(f"Batch {i}/{len(tiles)} | Time: {time.time()-start:.1f}s")
#     flush_to_db(pending_tables)
#     print(f"Pipeline done in {(time.time()-start)/60:.1f} min")
#
# run_pipeline()